In [17]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
#from pyspark.sql.functions import lower, col, explode, split, regexp_replace

In [18]:
spark_session = SparkSession.builder\
    .master("spark://192.168.2.70:7077") \
    .appName("Sepehr_3A")\
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", True)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.cores.max", 4)\
    .getOrCreate()

def count_special_words(path, special_words):
    sc = spark_session.sparkContext
    
    rdd = sc.textFile(path)
    words_rdd = rdd.flatMap(lambda line: line.split()).map(lambda word: word.lower())
    special_words_rdd = words_rdd.filter(lambda word: word in special_words).map(lambda word: (word, 1))
    counts_rdd = special_words_rdd.reduceByKey(lambda x, y: x + y)
    counts_dict = dict(counts_rdd.collect())
    
    sc.stop()
    
    return counts_dict




In [19]:
count_special_words("hdfs://192.168.2.70:9000/europarl/europarl-v7.sv-en.en", ['is', 'are'])

{'is': 758087, 'are': 337454}